In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils

In [2]:
events = utils.get_events()

In [3]:
events.head(1)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,month,day,hora
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0


### feature_top_model
La idea es asignar True/False si el modelo más frecuente relacionado al user es uno de los tops (mas viewed/checkout/conversion)

In [4]:
por_modelo = events.groupby(['model','event']).agg({'event':'count'}).unstack(fill_value=0)
por_modelo.columns = por_modelo.columns.droplevel(level=0).astype(object)
q=0.9
porcentaje = round((1-q)*100)
limite_viewed = por_modelo['viewed product'].quantile(q)
limite_conversion = por_modelo.conversion.quantile(q)
limite_checkout = por_modelo.checkout.quantile(q)
#print("Para estar en el top {}% de checkouts un modelo requiere tener {} eventos checkout".format(porcentaje,round(limite_checkout)))
#print("Para estar en el top {}% de conversions un modelo requiere tener {} eventos conversion".format(porcentaje,round(limite_conversion)))
#print("Para estar en el top {}% de viewed un modelo requiere tener {} eventos viewed".format(porcentaje,round(limite_viewed)))
top_viewed = por_modelo['viewed product'] > limite_viewed
top_checkout = por_modelo.checkout > limite_checkout
top_conversion = por_modelo.conversion > limite_conversion
top_modelos = por_modelo.loc[top_viewed | top_checkout | top_conversion]
top_modelos = set(top_modelos.index)
#print("Modelos Top: {}".format(top_modelos))

In [11]:
frecuencia_modelos_por_persona = events.groupby(['person']).model.value_counts().unstack(fill_value=0)
frecuencia_modelos_por_persona['most_frequent_model'] = frecuencia_modelos_por_persona.idxmax(axis=1)
features = pd.DataFrame(data=frecuencia_modelos_por_persona.most_frequent_model,columns=['most_frequent_model'])
features['is_most_frequent_model_top'] = features.most_frequent_model.isin(top_modelos)
features.head()

,most_frequent_model,is_most_frequent_model_top
person,,
0008ed71,LG G4 H818P,False
00091926,iPhone 6S,True
00091a7a,iPhone 6,True
000ba417,Samsung Galaxy A3 Duos,False
000c79fe,iPhone 7,True


In [12]:
#convierto las ciudades a enteros y los bool a 0/1
features.most_frequent_model = features.most_frequent_model.astype('category').cat.codes
features['is_most_frequent_model_top'] = features['is_most_frequent_model_top'].astype(int)

In [13]:
features.to_csv("features_top_model.csv")
features.head()

,most_frequent_model,is_most_frequent_model_top
person,,
0008ed71,14,0
00091926,166,1
00091a7a,164,1
000ba417,59,0
000c79fe,168,1
